In [46]:
import re
import glob
import nltk 

# Load Data

In [47]:
def load_data(filename, labels_filename):
    file = open(filename, 'r')
    labels_file = open(labels_filename, 'r')
    tweets = file.read()
    labels = labels_file.read()
    documents = tweets.split('\n')
    labels = labels.split('\n')
    return documents, labels

In [48]:
documents, labels = load_data('data/mex_train.txt', 'data/mex_train_labels.txt')
val_documents, val_labels = load_data('data/mex_val.txt', 'data/mex_val_labels.txt')
documents.pop(-1)
val_documents.pop(-1)
labels.pop(-1)
val_labels.pop(-1)

''

In [49]:
documents[:10]

['lo peor de todo es que no me dan por un tiempo y luego vuelven estoy hasta la verga de estl',
 'a la vga no seas mamón 45 putos minutos después me dices que apenas sales no me querías avisar en 3 horas? 😑',
 'considero que lo más conveniente seria que lo retes a unos vergazos mi jelipe! rómpele la madre a ese pinchi joto!',
 'el marica de mi ex me tiene bloqueada de todo así  uno no puede admirar la "belleza" de su garnacha 😂',
 'mujer despechadaya pinche amlo hazle esta que se pela la loca #reynosafollow #reynosa',
 'putos. no tienen madre. ambriados mantenidos. ojetes. como es posible. mejor matarlos',
 'ustedes si puden andar de chanceros pero cuidadito y seamos nosotras porque luego luego empiezan a mamar hijos de la chingada.',
 '@usuario jajjaja te digo esa madre si está buena ajjaja',
 'odio los putos trámites de titulación 😡😡😡😡😡😡😡😡😡😡😡 pero me urge la precedula.',
 '@usuario no te equivocabas mi madre y tu tenían muchísima razón siempre es mejor lo que viene 💚']

In [50]:
len(documents) == len(labels)

True

In [51]:
val_documents[:10]

['así debería ser siempre para que se mueran a la verga',
 'cada día me siento como un perro ovejero tratando de cuidar sus ovejas vale madre pinches rateros póngase a trabajar.😤',
 'hijo de tu puta madre nadamas te la pasas mamando pinche wilo de closet',
 'soñé horrible espero no se cumpla putos temblores.',
 '@usuario mejor vas y la chingas tú veo tus publicaciones y solo denotan inconformidad como típico ciudadano quejumbroso que no hace nada por cambiar',
 'y quizás este loca y quizás me guste estarlo... #siempreneruda',
 'me tienen hasta la verga con sus putos #boomerang pendejos y sin sentido.',
 'cuando va a ser el día que encuentre a alguien en quien pueda  confiar posts la re puta madre?',
 '#ahoritaestoypensando  como putas puede pasar esto?',
 '#todoibabienpero valio madre ya me atacó el insomnio🙅']

In [52]:
len(val_documents) == len(val_labels)

True

In [63]:
from nltk import TweetTokenizer
from nltk.corpus import stopwords

def process_documents(documents):
    # tokenize each document
    documents_tokenized = []
    tokenizer = TweetTokenizer()
    for doc in documents:
        documents_tokenized.append(tokenizer.tokenize(doc.lower()))
    return documents_tokenized

def remove_stop_words(documents):
    # build dictionary of stopwords
    stopwords_dict = {word:1 for word in stopwords.words('spanish')}
    non_stop_documents = []
    for doc in documents:
        ndoc = []
        for word in doc:
            if stopwords_dict.get(word) == None:
                ndoc.append(word)
        non_stop_documents.append(ndoc)
    
    return non_stop_documents

In [54]:
documents = process_documents(documents)
val_documents = process_documents(val_documents)

# Information Gain

In [ ]:
# compute information gain between each token and classes
# store value in dictionary for each word
# remove all the ones that are below threshold
# assing indexes

# BoW Class

In [55]:
from nltk import TweetTokenizer
from nltk.probability import FreqDist
import numpy as np

class BoWBuilder:
    # UTILITIES
    def get_dimensions(self):
        return self.voc_index.keys()
    
    # INIT FUNCTIONS
    def __init__(self):
        # attributes
        self.voc_index = {} 
        self.train_idf = None
        self.T = 0
        
    def get_vocabulary(self, documents, T):
        # get vocabulary
        tokens = [token for doc in documents for token in doc]
        vocabulary = FreqDist(tokens)
        
        self.T = min(T, len(vocabulary.keys()))
        
        # get most common words
        limited_voc = vocabulary.most_common(T)
        self.voc_index = {}
        
        # get index of words in matrix
        for i, word_count in enumerate(limited_voc):
            self.voc_index[word_count[0]] = i
    
    
    # BUILD BOW MATRIX
    def build_bow(self, documents, T=5000, voc_index=None, mode='train', weight_scheme='binary', normalize=False):
        # get most common terms - training mode
        if mode == 'train':
            if voc_index==None:
                self.get_vocabulary(documents, T) #use most common words as vocabulary
            else:
                # use vocabulary index sent as parameter. Usefull when performing a features reduction or working with n-grams
                self.voc_index = voc_index
                self.T = len(voc_index.keys())
        
        # use train_idf, testing mode
        use_train_idf = mode != 'train'
        
        # get weights for matrix
        if weight_scheme == 'tf':
            bow = self.frequency_bow(documents)
        elif weight_scheme == 'tf-idf':
            # if documents!= None, use existing idf weights (val or test mode)
            bow = self.frequency_bow(documents, use_idf=True, use_train_idf=use_train_idf)
        else:
            bow = self.binary_bow(documents)
        
        # normalize if necessary
        if normalize:
            norm = np.linalg.norm(bow, axis=1)
            # Add 1 if norm == 0 to avoid division by 0. --  Increase 1 dimension for broadcast 
            bow = bow / (norm + (norm==0 + 0.0))[:, np.newaxis]
        
        return bow
            
    # WEIGHT SCHEMES
    def binary_bow(self, documents):
        N = len(documents)
        T = self.T
        
        bow = np.zeros((N, T))
        for i, doc in enumerate(documents):
            for word in doc:
                j = self.voc_index.get(word)
                if j != None:
                    bow[i, j] = 1 
        
        return bow
    
    def frequency_bow(self, documents, use_idf=False, use_train_idf=False):
        N = len(documents)
        T = self.T
        bow = np.zeros((N, T))
        
        # tf scheme
        for i, doc in enumerate(documents):
            for word in doc:
                j = self.voc_index.get(word)
                if j != None:
                    bow[i, j] += 1 
        
        # tf-idf scheme
        if use_idf:
            if not use_train_idf:
                # calculate idf for first time (training mode)
                self.train_idf = np.sum(bow>0, axis=0)
            
            idf = self.train_idf     
            bow = np.log(bow + 1) * np.log(N/idf)
            
        return bow        

# Bolsas de Palabras, Bigramas y Emociones (40pts)

**<h3>Model and Labels (EXECUTE FIRST!!)</h3>**

In [56]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support

def get_model(max_iter=1000):
    parameters = {'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]}
    svr = svm.LinearSVC(class_weight='balanced', max_iter=max_iter)
    grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring='f1_macro', cv=5) 
    return grid

def get_nl_model(max_iter=1000):
    parameters = {'C': [0.05, 0.12, 0.25, 0.5, 1, 2, 4]}
    svr = svm.SVC(kernel='poly', class_weight='balanced', max_iter=max_iter)
    grid = GridSearchCV(estimator=svr, param_grid=parameters, n_jobs=8, scoring='f1_macro', cv=5) 
    return grid

In [97]:
# labels
y_train = list(map(int, labels))
y_val = list(map(int, val_labels))

# BoW Builder
bow_builder = BoWBuilder()

def train_and_test(model, bow, val_bow):
    model.fit(bow, y_train)
    pred = model.predict(val_bow)
    p, r, f, _ = precision_recall_fscore_support(y_val, pred, average='macro', pos_label=None)
    a = accuracy_score(y_val, pred)
    print(" accuracy: ", a, "\n precision: ", p, "\n recall: ", r, "\n f_measure: ", f)

In [58]:
"ratio of positive labels: ", sum(y_train)/len(y_train)

('ratio of positive labels: ', 0.3573232323232323)

<h3> 1. Evalué BoW con pesado Binario </h3>

In [98]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=5000, weight_scheme='binary', normalize=False)
val_bow = bow_builder.build_bow(val_documents, T=5000, mode='test', weight_scheme='binary', normalize=False)
model = get_model(2000)
train_and_test(model, bow, val_bow)

 accuracy:  0.8133116883116883 
 precision:  0.7958333333333334 
 recall:  0.807051746546588 
 f_measure:  0.8003421578491969


<h3> 2. Evalué BoW con pesado de Frecuencia </h3>

In [99]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=5000, weight_scheme='tf', normalize=False)
val_bow = bow_builder.build_bow(val_documents, T=5000, mode='test', weight_scheme='tf', normalize=False)
model = get_model(5000)
train_and_test(model, bow, val_bow)

 accuracy:  0.8181818181818182 
 precision:  0.800838198545479 
 recall:  0.8087827657200695 
 f_measure:  0.8043113228953936


<h3> 3. Evalué BoW con pesado tf-idf </h3>

In [100]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=5000, weight_scheme='tf-idf', normalize=False)
val_bow = bow_builder.build_bow(val_documents, T=5000, mode='test', weight_scheme='tf-idf', normalize=False)
model = get_model(15000)
train_and_test(model, bow, val_bow)

 accuracy:  0.6948051948051948 
 precision:  0.7023890612780342 
 recall:  0.5922730984668116 
 f_measure:  0.578082191780822


<h3> 4. Evalué BoW con pesado Binario Normalizado l2 </h3>

In [101]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=5000, weight_scheme='binary', normalize=True)
val_bow = bow_builder.build_bow(val_documents, T=5000, mode='test', weight_scheme='binary', normalize=True)
model = get_model(2000)
train_and_test(model, bow, val_bow)

 accuracy:  0.7987012987012987 
 precision:  0.7809010396611475 
 recall:  0.7936694155941248 
 f_measure:  0.785650143678161


<h3> 5. Evalué BoW con pesado Frecuencia Normalizado l2 </h3>

In [102]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=5000, weight_scheme='tf', normalize=True)
val_bow = bow_builder.build_bow(val_documents, T=5000, mode='test', weight_scheme='tf', normalize=True)
model = get_model(5000)
train_and_test(model, bow, val_bow)

 accuracy:  0.7954545454545454 
 precision:  0.7775125144397381 
 recall:  0.7901268647274651 
 f_measure:  0.7821928879310345


<h3> 6. Evalué BoW con pesado tf-idf Normalizado l2 </h3>

In [103]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=5000, weight_scheme='tf-idf', normalize=True)
val_bow = bow_builder.build_bow(val_documents, T=5000, mode='test', weight_scheme='tf-idf', normalize=True)
model = get_model(15000)
train_and_test(model, bow, val_bow)

 accuracy:  0.7256493506493507 
 precision:  0.7279454651501895 
 recall:  0.6438413673326202 
 f_measure:  0.6482200483214219


<h3> 7. Ponga una tabla comparativa a modo resumen con las 6 entradas anteriores. </h3>

**<h2> Tabla de Resultados </h2>**

A continuación mostramos los resultados de la métrica de accuracy resumidos en una tabla para cada modelo.

**No.** | **Modelo** | **accuracy** | **precision** | **recall**
 -------- |----| ---- | ---- | ----
1 |`Binary` | 0.7881  | 0.5595 | 0.5600
2 | `TF` | 0.7818 | 0.7726 | 0.7550
3 | `TF-IDF` | 0.7407 | 0.7267 | 0.7086
4 | `Normalized Binary` | 0.7564 | 0.7423 | 0.7221
5 | `Normalized TF` | 0.9211 | 0.9060 | 0.8032
5 | `Normalized TF-IDF` | 0.9211 | 0.9060 | 0.8032

<h3> 8. De las configuraciones anteriores elija la mejor y evalúela con más y menos términos
(e.g., 1000 y 7000). Ponga una tabla dónde compare las tres configuraciones. </h3>

In [104]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=1000, weight_scheme='tf', normalize=False)
val_bow = bow_builder.build_bow(val_documents, T=1000, mode='test', weight_scheme='tf', normalize=False)
model = get_model(15000)
train_and_test(model, bow, val_bow)

 accuracy:  0.8214285714285714 
 precision:  0.8042776567822638 
 recall:  0.8133489757657315 
 f_measure:  0.8081605381470618


In [105]:
# get bows, and model
bow = bow_builder.build_bow(documents, T=7000, weight_scheme='tf', normalize=False)
val_bow = bow_builder.build_bow(val_documents, T=7000, mode='test', weight_scheme='tf', normalize=False)
model = get_model(15000)
train_and_test(model, bow, val_bow)

 accuracy:  0.8181818181818182 
 precision:  0.8008658008658008 
 recall:  0.8077591065410672 
 f_measure:  0.8039443023586246


<h3> 9. Utilice el recurso léxico del Consejo Nacional de Investigación de Canadá llamado
"EmoLex" (https://www.saifmohammad.com/WebPages/NRC-Emotion-Lexicon.htm) para
construir una "Bolsa de Emociones" de los Tweets de agresividad (Debe usar EmoLex
en Español). Para esto, una estrategia sencilla sería enmascarar cada palabra con su
emoción, y después construir la Bolsa de Emociones (BoE). </h3

In [91]:
import pandas
emotion_data = pandas.read_excel('data/emolex.xlsx', usecols="CI,DB:DK")

emotion_dict = {}
for i in range(len(emotion_data)):
    row = emotion_data.loc[i]
    mask = ''
    for val in row[1:]:
        mask+= str(val)
    emotion_dict[row[0]] = mask

In [92]:
emotion_dict['disfrutar']

'1001001001'

In [93]:
def mask_word(mask_dict, documents, ignore=True):
    masked_documents = []
    for doc in documents:
        masked_doc = []
        for word in doc:
            mask = mask_dict.get(word)
            if mask != None:
                masked_doc.append(mask)
            elif not ignore:
                masked_doc.append(word)
                
        masked_documents.append(masked_doc)
    
    return masked_documents

In [94]:
emolex_documents = mask_word(emotion_dict, documents, ignore=False)
emolex_val_documents = mask_word(emotion_dict, val_documents, ignore=False)

# BoWBuilder object
bow_builder = BoWBuilder()

In [199]:
emolex_bow = bow_builder.build_bow(documents, T=2048, weight_scheme='binary')
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, T=2048, mode='test', weight_scheme='binary')

<h3>10. Evalúa tú BoE clasificando con SVM. Ponga una tabla comparativa a modo de resumen
con los tres pesados, normalize cada uno si lo cree conveniente. </h3>

In [106]:
emolex_bow = bow_builder.build_bow(emolex_documents, T=2048, weight_scheme='binary')
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, mode='test', weight_scheme='binary')
model = get_model(5000)
train_and_test(model, emolex_bow, val_emolex_bow)

 accuracy:  0.7873376623376623 
 precision:  0.7682203389830509 
 recall:  0.7766640212553052 
 f_measure:  0.7717453717453717


In [107]:
emolex_bow = bow_builder.build_bow(emolex_documents, T=2048, weight_scheme='tf')
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, mode='test', weight_scheme='tf')
model = get_model(13000)
train_and_test(model, emolex_bow, val_emolex_bow)

 accuracy:  0.8003246753246753 
 precision:  0.7818834742918512 
 recall:  0.7898105655429419 
 f_measure:  0.7852910421749739


In [109]:
emolex_bow = bow_builder.build_bow(emolex_documents, T=2048, weight_scheme='tf-idf')
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, mode='test', weight_scheme='tf-idf')
model = get_model(15000)
train_and_test(model, emolex_bow, val_emolex_bow)

/home/juan/anaconda3/envs/nlp/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/juan/anaconda3/envs/nlp/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/home/juan/anaconda3/envs/nlp/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


 accuracy:  0.702922077922078 
 precision:  0.6820700869449792 
 recall:  0.6262091255190182 
 f_measure:  0.6289779515585967


**<h4>Normalized</h4>**

In [110]:
emolex_bow = bow_builder.build_bow(emolex_documents, T=2048, weight_scheme='binary', normalize=True)
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, mode='test', weight_scheme='binary', normalize=True)
model = get_model(5000)
train_and_test(model, emolex_bow, val_emolex_bow)

 accuracy:  0.7873376623376623 
 precision:  0.7683629007158419 
 recall:  0.7776876804343076 
 f_measure:  0.7721578315804334


In [111]:
emolex_bow = bow_builder.build_bow(emolex_documents, T=2048, weight_scheme='tf', normalize=True)
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, mode='test', weight_scheme='tf', normalize=True)
model = get_model(5000)
train_and_test(model, emolex_bow, val_emolex_bow)

 accuracy:  0.788961038961039 
 precision:  0.7700032285717466 
 recall:  0.7789471262781362 
 f_measure:  0.7736932537529391


In [112]:
emolex_bow = bow_builder.build_bow(emolex_documents, T=2048, weight_scheme='tf-idf', normalize=True)
val_emolex_bow = bow_builder.build_bow(emolex_val_documents, mode='test', weight_scheme='tf-idf', normalize=True)
model = get_model(13000)
train_and_test(model, emolex_bow, val_emolex_bow)

 accuracy:  0.7159090909090909 
 precision:  0.6922965051237302 
 recall:  0.697704243009788 
 f_measure:  0.6945197754521986


**<h2> Tabla de Resultados </h2>**

A continuación mostramos los resultados de la métrica de accuracy resumidos en una tabla para cada modelo.

**No.** | **Modelo** | **accuracy** | **precision** | **recall**
 -------- |----| ---- | ---- | ----
1 |`Binary` | 0.7881  | 0.5595 | 0.5600
2 | `TF` | 0.7818 | 0.7726 | 0.7550
3 | `TF-IDF` | 0.7407 | 0.7267 | 0.7086
4 | `Normalized Binary` | 0.7564 | 0.7423 | 0.7221
5 | `Normalized TF` | 0.9211 | 0.9060 | 0.8032
5 | `Normalized TF-IDF` | 0.9211 | 0.9060 | 0.8032

# 3 Recurso Línguistico de Emociones Mexicano (30 pts)

<h3> 1. Utilice el recurso léxico llamado "Spanish Emotion Lexicon (SEL)" del Dr. Grigori Sidorov, profesor del Centro de Investigación en Computación (CIC) del Instituto Politécnico Nacional (http://www.cic.ipn.mx/∼sidorov/), para enmascarar cada palabra con su emoción, y después construir la Bolsa de Emociones con algún pesado (e.g., binario, tf, tfidf). Proponga alguna estrategia para incorporar el "valor" del "Probability Factor of Affective use" en su representación vectorial del documento. Evalúa y escribe una tabla comparativa a modo de resumen con al menos tres pesados: binario, frecuencia, tfidf. Normalize cada pesado según lo crea conveniente de acuerdo el experimento (1). </h3>

In [113]:
import pandas
emotion_data = pandas.read_excel('data/SEL.xlsx', sheet_name="Hoja1", usecols="B,C,D")

# build dictionary will all the words in sel resource
sel_dict = {}
for i in range(len(emotion_data)):
    row = emotion_data.loc[i]
    mask = row[2]
    pfa = row[1]
    sel_dict[row[0]] = (mask, pfa)

In [114]:
sel_dict['odio']

('Enojo', 0.932)

In [115]:
# mask word as a pair <word>-<emotion> 
# this is because we want to get specific pfa for each word with its emotion
def mask_word_sel(mask_dict, documents, ignore=True):
    masked_documents = []
    for doc in documents:
        masked_doc = []
        for word in doc:
            mask = mask_dict.get(word)
            if mask != None:
                masked_doc.append(word + ":::" + mask[0])
            elif not ignore:
                masked_doc.append(word)
        masked_documents.append(masked_doc)
    
    return masked_documents


'''
    -- build pfa matrix
    * voc_index: dictionary with the column position of each word in the bow
    * sel_dict: dictionary with the sel_dict resource to retrieve its pfa
    * documents: masked documents by mask_word function
    * return: pfa_matrix
'''

def get_pfa_matrix(voc_index, sel_dict, documents, default_weight=1):
    pfa_matrix = np.ones((len(documents), len(voc_index))) * default_weight
    for i, doc in enumerate(documents):
        for word_emo in doc:
            l_words = word_emo.split(":::")
            #word, emo = l_words[0], l_words[1] if len(l_words) == 2 else word_emo, None
            if len(l_words) == 2:
                word, emo = l_words
            else:
                word = word_emo
            emotion_pfa = sel_dict.get(word)
            if emotion_pfa != None:
                j = voc_index.get(word_emo)
                if j != None:
                    pfa_matrix[i, j] = emotion_pfa[1]
    
    return pfa_matrix

En el siguiente bloque construimos nuestra BoW enmascarando las palabras por el par \<palabra\>-\<emoción\> como se explicó anteriormente y luego realizamos la multiplicación de la BoW por su matriz PFA correspondiente. En este momento ignoramos aquellas palabras que no se encuentran en el recurso léxico SEL. 

In [116]:
# build masked documents
sel_documents = mask_word_sel(sel_dict, documents, ignore=True)
val_sel_documents = mask_word_sel(sel_dict, val_documents, ignore=True)

# bow builder object
bow_builder = BoWBuilder()

# build bow for masked documents and get its pfa matrix
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='binary')
sel_pfa = get_pfa_matrix(bow_builder.voc_index, sel_dict, sel_documents)

# build bow for masked validation documents and get its pfa matrix
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='binary')
val_sel_pfa = get_pfa_matrix(bow_builder.voc_index, sel_dict, val_sel_documents)

# compute final bows by element-wise-multiplication (original_bow * pfa_matrix)
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(5000)
train_and_test(model, sel_bow, val_sel_bow)

 accuracy:  0.6201298701298701 
 precision:  0.5409413281753708 
 recall:  0.5241019978606674 
 f_measure:  0.5040667189628835


Ahora, veamos que sucede cuando no ignoramos las palabras que no aparecen en el recurso léxico SEL.

In [117]:
# build masked documents
sel_documents = mask_word_sel(sel_dict, documents, ignore=False)
val_sel_documents = mask_word_sel(sel_dict, val_documents, ignore=False)

# bow builder object
bow_builder = BoWBuilder()

# build bow for masked documents and get its pfa matrix
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='binary')
sel_pfa = get_pfa_matrix(bow_builder.voc_index, sel_dict, sel_documents, default_weight=1)

# build bow for masked validation documents and get its pfa matrix
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='binary')
val_sel_pfa = get_pfa_matrix(bow_builder.voc_index, sel_dict, val_sel_documents, default_weight=1)

# compute final bows by element-wise-multiplication (original_bow * pfa_matrix)
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(5000)
train_and_test(model, sel_bow, val_sel_bow)

 accuracy:  0.810064935064935 
 precision:  0.7924202127659574 
 recall:  0.8035091956799283 
 f_measure:  0.796869847550922


Podemos ver que el rendimiento es mucho mejor de manera general, todas las métricas obtienen un aumento considerable. **¿Qué puede estar sucediendo?**. Una hipótesis es que al enmascarar las palabras por su emoción y multiplicar estas por su PFA se intensifica o atenua sus pesos en función de que tan fuerte es la emoción. Pero, esto ya se hacía en el bloque anterior. Entonces, ¿por qué aquí tenemos una aumento considerable? Evidentemente las palabras ignoradas anteriormente aportan información valiosa, y aunque podría parecer contra intuitivo debido a que las palabras que en principio se ignoran se sospecha que tienen una emoción neutral o carecen de emoción, estas palabras pueden de cierta manera caracterizar una transitividad de la  emoción. Por ejemplo, consideremos el siguiente par de oraciones:

1. Odio lo que ha sucedido.
2. Te odio!

Si observamos en el diccionario del recurso léxico SEL, podemos ver que la palabra odio aparece con una emoción de enojo y un PFA superior a 0.9. Sin embargo, al ignorar las demás palabras omitimos información importante, porque tal vez los tweets que tengan palabras como "te", "le", "la", "lo", etc. denotan que se hace referencia a una persona probablemente. Y estos tweets tal vez estan catalogados como tweets de agresividad. Mientras que los tweets que tienen la palabra "odio" pero no tienen las otras palabras tal vez solo expresan una emoción y no agresividad hacia alguien. Y el dataset puede contener casos que ejemplifican esto, entonces esta información termina siendo relevante. 

El ejemplo de arriba puede no ser el mejor para el caso específico de agresividad, pero ilustra el punto de porqué esta información puede ser valiosa aunque no se haga como tal una interpretación semántica, pero estadísticamente puede marcar una diferencia.

Por lo tanto, decidimos en los siguientes bloques proseguir sin ignorar las palabras que no aparecen en el recurso léxico, y en general en todos se observó una mejora.

In [118]:
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='tf')
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='tf')
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(5000)
train_and_test(model, sel_bow, val_sel_bow)

 accuracy:  0.8246753246753247 
 precision:  0.8077410100964826 
 recall:  0.815867867453389 
 f_measure:  0.8113002042205582


In [119]:
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='tf-idf')
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='tf-idf')
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(15000)
train_and_test(model, sel_bow, val_sel_bow)

/home/juan/anaconda3/envs/nlp/lib/python3.9/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


 accuracy:  0.6948051948051948 
 precision:  0.7050264892104923 
 recall:  0.5912494392878093 
 f_measure:  0.5760743831905704


In [121]:
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='binary', normalize=True)
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='binary', normalize=True)
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(5000)
train_and_test(model, sel_bow, val_sel_bow)

 accuracy:  0.8003246753246753 
 precision:  0.7825004406839415 
 recall:  0.7949288614379536 
 f_measure:  0.7871966341522327


In [122]:
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='tf', normalize=True)
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='tf', normalize=True)
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(5000)
train_and_test(model, sel_bow, val_sel_bow)

 accuracy:  0.7938311688311688 
 precision:  0.7750892060660125 
 recall:  0.7837491229886246 
 f_measure:  0.7787149787149787


In [124]:
sel_bow = bow_builder.build_bow(sel_documents, T=5000, weight_scheme='tf-idf', normalize=True)
val_sel_bow = bow_builder.build_bow(val_sel_documents, mode='test', weight_scheme='tf-idf', normalize=True)
sel_bow = sel_bow * sel_pfa
val_sel_bow = val_sel_bow * val_sel_pfa

model = get_model(5000)
train_and_test(model, sel_bow, val_sel_bow)

 accuracy:  0.724025974025974 
 precision:  0.7243182599569332 
 recall:  0.6425819214887915 
 f_measure:  0.646827109864019


**<h2> Tabla de Resultados </h2>**

A continuación mostramos los resultados de la métrica de accuracy resumidos en una tabla para cada modelo.

**No.** | **Modelo** | **accuracy** | **precision** | **recall**
 -------- |----| ---- | ---- | ----
1 |`Binary` | 0.7881  | 0.5595 | 0.5600
2 | `TF` | 0.7818 | 0.7726 | 0.7550
3 | `TF-IDF` | 0.7407 | 0.7267 | 0.7086
4 | `Normalized Binary` | 0.7564 | 0.7423 | 0.7221
5 | `Normalized TF` | 0.9211 | 0.9060 | 0.8032
5 | `Normalized TF-IDF` | 0.9211 | 0.9060 | 0.8032

<h3> 2. En un comentario aparte, discuta sobre la estrategía que utilizó para incorporar el
"Probability Factor of Affective use". No más de 5 renglones. </h3> 

Primero se generó una matriz de pares de la forma \<word\>-\<emotion\> y con esto se construyo una bolsa de palabras. Esto con la intención de que palabras diferentes con la misma emoción generaran un par distinto para poder usar sus valores de PFA. Luego, se construyó una matriz de los valores de PFA para cada palabra que existía en el recurso léxico y se encontraba en los documentos. De esta manera tanto la BoW como la matriz PFA tiene las mismas dimensiones. Aquellas palabras que estaban en el recurso léxico se quedaron con un valor del parámetro **default_weight** en esta matriz. Finalmente se procedió a realizar una multiplicación punto a punto entre ambas matrices con el objetivo de que aquellas palabras que tengan un PFA bajo tengan menor relevancia que aquellas que tengan un PFA alto. 

# 4  ¿Podemos mejorar con bigramas?

<h3> 1. Hacer un experimento dónde concatene una buena BoW según sus experimentos anteriores con otra BoW construida a partir de los 1000 bigramas más frecuentes. </h3>

In [167]:
all_words = [word for doc in documents for word in doc]

In [168]:
from nltk.collocations import *
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(all_words)

#finder.apply_freq_filter(10)
bigrams = finder.nbest(bigram_measures.pmi, 1000)

In [176]:
bigram_dict = {}
for i, bigram in enumerate(bigrams):
    bigram_dict[bigram[0]+bigram[1]] = i

In [181]:
def build_bigram_document(documents):
    bigram_documents = []
    for doc in documents:
        bigram_doc = []
        for i in range(len(doc)-1):
            bigram_doc.append(doc[i] + doc[i+1])
        bigram_documents.append(bigram_doc)
    
    return bigram_documents

In [182]:
bigram_documents = build_bigram_document(documents)
val_bigram_documents = build_bigram_document(val_documents)

In [183]:
bow_builder = BoWBuilder()
bigram_bow = bow_builder.build_bow(bigram_documents, voc_index=bigram_dict, weight_scheme='binary')
val_bigram_bow = bow_builder.build_bow(val_bigram_documents, mode='test', weight_scheme='binary')

In [188]:
bigram_bow = np.column_stack((sel_bow, bigram_bow))
val_bigram_bow = np.column_stack((val_sel_bow, val_bigram_bow))

In [189]:
model = get_model(5000)
model.fit(bigram_bow, y_train)
pred = model.predict(val_bigram_bow)
p, r, f, _ = precision_recall_fscore_support(y_val, pred, average='macro', pos_label=None)
a = accuracy_score(y_val, pred)
print(" accuracy: ", a, "\n precision: ", p, "\n recall: ", r, "\n f_measure: ", f)

 accuracy:  0.8198051948051948 
 precision:  0.8026244649695877 
 recall:  0.8090185523848958 
 f_measure:  0.8055107561289881


<h3> 2. Hacer un experimento con las Bolsas de Emociones, Bolsa de Palabras y Bolsa de Bi-
gramas; usted elige las dimensionalidades. Para construir la representación final del
documento utilice la concatenación de las representaciones según sus observaciones
(e.g., Bolsa de Palabras + Bolsa de Bigramas + Bolsa de Sentimientos de Canadá + Bolsa
de Sentimientos de Grigori), y aliméntelas a un SVM. </h3>

<h3> 3. Elabore conclusiones sobre toda esta Tarea, incluyendo observaciones, comentarios y
posibles mejoras futuras. Discuta el comportamiento de la BoW de usar solo palabras
a integrar bigramas, y luego a integrar todo ¿ayudó? o ¿empeoró?. Discuta también
brevemente el costo computacional de los experimentos ¿Valió la Pena tener todo?. Sea
breve: todo en NO más de dos párrafos. </h3>

# Pruebas

In [120]:
docs = ["hola como estas", 
    "hola mundo, voy a comer el mundo",
    "hola me voy a ir",
    "hola quiero comer",
    "hola voy a comer hoy",
    "hola el mundo esta mal"]

In [121]:
bow_object = BoW(docs)
bow = bow_object.build_bow(T=3, weight_scheme='tf-idf', normalize=True)

In [122]:
bow

array([[0.        , 0.        , 0.        ],
       [0.        , 0.92909298, 0.36984623],
       [0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        ],
       [0.        , 1.        , 0.        ]])

In [69]:
bow_object.get_dimensions()

[('hola', 3), ('mundo', 3), ('voy', 3)]